Team Comparisons

In [2]:
import csv
import numpy as np
import pandas as pd

Import Data

In [3]:
Stats = []
with open('TI9.csv', newline = '') as csvfile:
    spamreader = csv.reader(csvfile, delimiter = ',', quotechar='|')
    for row in spamreader:
        Stats.append(row)

Games Array

In [4]:
Team1 = []
Team2 = []
Winner = []
i = 0
for game in Stats:
    if i == 0:
        i += 1
    else:
        Team1.append(Stats[i][1])
        Team2.append(Stats[i][4])
        Winner.append(Stats[i][7])
        i += 1       

Total Wins per team

In [5]:
Teamlist = ['OG','Team Liquid', 'PSG.LGD','Team Secret','Vici Gaming','Evil Geniuses','Royal Never Give Up','Infamous Gaming', 'Newbee', 'TNC Predator','Virtus.pro','Fnatic','Alliance','Mineski','Natus Vincere', 'Chaos Esports Club', 'KEEN GAMING', 'Ninjas in Pyjamas']


wij = np.zeros((len(Teamlist), len(Teamlist)))


wincounts = []
for x in Teamlist:
    print(x)
    wincounts.append(Winner.count(x))

OG
Team Liquid
PSG.LGD
Team Secret
Vici Gaming
Evil Geniuses
Royal Never Give Up
Infamous Gaming
Newbee
TNC Predator
Virtus.pro
Fnatic
Alliance
Mineski
Natus Vincere
Chaos Esports Club
KEEN GAMING
Ninjas in Pyjamas


Wins i vs j matrix

In [7]:
d = {'Radiant':Team1, 'Dire':Team2, 'Winner':Winner}

index_dict = {'OG':0,'Team Liquid':1,'PSG.LGD':2,'Team Secret':3,'Vici Gaming':4,'Evil Geniuses':5,'Royal Never Give Up':6,'Infamous Gaming':7,'Newbee':8,'TNC Predator':9,'Virtus.pro':10,'Fnatic':11,'Alliance':12,'Mineski':13,'Natus Vincere':14,'Chaos Esports Club':15,'KEEN GAMING':16,'Ninjas in Pyjamas':17}

games = pd.DataFrame(data = d)

Wi = pd.DataFrame(np.array([wincounts]), columns = Teamlist)
    
np.array(wincounts)


for n in range(len(games.loc[:,'Radiant'])):
    i = games.loc[n,'Radiant']
    j = games.loc[n,'Dire']
    if games.loc[n,'Winner'] == i:
        wij[index_dict[i],index_dict[j]] += 1
    else:
        wij[index_dict[j],index_dict[i]] += 1

print(wij)

[[0. 6. 4. 0. 2. 8. 4. 4. 4. 0. 4. 4. 0. 0. 2. 0. 0. 4.]
 [2. 0. 4. 4. 0. 4. 4. 0. 0. 4. 0. 2. 2. 2. 0. 4. 4. 0.]
 [2. 6. 0. 4. 4. 0. 0. 0. 2. 2. 4. 0. 2. 4. 0. 4. 4. 0.]
 [0. 4. 0. 0. 4. 2. 0. 4. 4. 2. 0. 0. 4. 6. 0. 2. 4. 0.]
 [2. 0. 0. 0. 0. 0. 4. 2. 0. 4. 4. 4. 0. 0. 4. 0. 0. 2.]
 [2. 0. 0. 4. 4. 0. 4. 2. 0. 0. 2. 0. 0. 0. 2. 0. 0. 4.]
 [0. 0. 0. 0. 0. 0. 0. 2. 0. 0. 6. 0. 2. 0. 4. 0. 0. 4.]
 [0. 0. 0. 0. 2. 2. 2. 0. 4. 0. 0. 4. 0. 0. 2. 0. 2. 2.]
 [0. 4. 2. 0. 0. 0. 0. 2. 0. 2. 0. 0. 2. 2. 0. 4. 2. 0.]
 [0. 4. 2. 2. 2. 0. 0. 0. 2. 0. 0. 0. 2. 0. 0. 2. 4. 0.]
 [0. 0. 0. 0. 0. 2. 2. 4. 0. 0. 0. 4. 0. 0. 0. 0. 0. 4.]
 [0. 0. 0. 0. 0. 4. 4. 0. 0. 0. 0. 0. 0. 0. 4. 0. 0. 2.]
 [0. 2. 2. 0. 0. 0. 0. 0. 2. 2. 0. 0. 0. 2. 0. 2. 4. 0.]
 [0. 2. 0. 4. 0. 0. 0. 0. 2. 4. 0. 0. 2. 0. 2. 4. 0. 0.]
 [2. 0. 0. 0. 0. 2. 0. 2. 0. 0. 4. 0. 0. 0. 0. 0. 0. 4.]
 [0. 0. 0. 2. 0. 0. 0. 0. 0. 2. 0. 0. 2. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 2. 0. 0. 0. 0. 4. 0. 4. 0. 0.]
 [0. 0. 0. 0. 2. 0. 0. 2. 0. 0.

To implement the Bradley Terry Model as a GLM we will be relying on the logistic model for binomially distributed data as implemented by Jamil, 2010 such that logit(p_(i>j))= log(p_(i>j)/(1-p_(i>j)) = lamda_i - lambda_j


Model Matrix

In [96]:
"""Number of teams"""
k = len(Teamlist)
tmp = np.zeros((k,k), dtype=int)
X = np.zeros((int(k*(k-1)/2), k))

In [105]:
""" Design Matrix X"""
j = 0
i = -1
k = len(Teamlist)
for row in X:
    if j%(k) == 0:
        i += 1
        j = i+1
    row[i] = 1
    row[j] = -1
    print("row ("+str(i)+", "+str(j)+")")
    print(row)
    j += 1


    
    


row (0, 1)
[ 1. -1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
row (0, 2)
[ 1.  0. -1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
row (0, 3)
[ 1.  0.  0. -1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
row (0, 4)
[ 1.  0.  0.  0. -1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
row (0, 5)
[ 1.  0.  0.  0.  0. -1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
row (0, 6)
[ 1.  0.  0.  0.  0.  0. -1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
row (0, 7)
[ 1.  0.  0.  0.  0.  0.  0. -1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
row (0, 8)
[ 1.  0.  0.  0.  0.  0.  0.  0. -1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
row (0, 9)
[ 1.  0.  0.  0.  0.  0.  0.  0.  0. -1.  0.  0.  0.  0.  0.  0.  0.  0.]
row (0, 10)
[ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0. -1.  0.  0.  0.  0.  0.  0.  0.]
row (0, 11)
[ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. -1.  0.  0.  0.  0.  0.  0.]
row (0, 12)
[ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. -1

In [104]:
X

array([[ 1., -1.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0., -1., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       ...,
       [ 0.,  0.,  0., ...,  1., -1.,  0.],
       [ 0.,  0.,  0., ...,  1.,  0., -1.],
       [ 0.,  0.,  0., ...,  0.,  1., -1.]])

In [166]:
""" Probability Matrix """
Pij = np.zeros((18,18))
Pij

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.],
       [0., 0., 0., 0

In [167]:
for i in range(k):
    print(Pij[i-1])
    for j in range(k):
        if i != j:
            if wij[i,j] != 0 and wij[j,i] != 0:
                Pij[i,j] = wij[i,j]/(wij[i,j] + wij[j,i])

"""We treat no comparisons as 0% win probability though this assumption should probably be viewed more"""

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0.         0.75       0.66666667 0.         0.5        0.8
 0.         0.         0.         0.         0.         0.
 0.         0.         0.5        0.         0.         0.        ]
[0.25 0.   0.4  0.5  0.   0.   0.   0.   0.   0.5  0.   0.   0.5  0.5
 0.   0.   0.   0.  ]
[0.33333333 0.6        0.         0.         0.         0.
 0.         0.         0.5        0.5        0.         0.
 0.5        0.         0.         0.         0.         0.        ]
[0.         0.5        0.         0.         0.         0.33333333
 0.         0.         0.         0.5        0.         0.
 0.         0.6        0.         0.5        0.         0.        ]
[0.5        0.         0.         0.         0.         0.
 0.         0.5        0.         0.66666667 0.         0.
 0.         0.         0.         0.         0.         0.5       ]
[0.2        0.         0.         0.66666667 0.         0.
 0.         0.5        0.         0.    

From here we should be able to apply the logit model as we have pij, X and unknowns lambda where
logit[pij] = X*Lambda